In [1]:
from convokit import Corpus, download

In [2]:
corpus = Corpus(filename=download('subreddit-Cornell'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/subreddit-Cornell


In [3]:
corpus.print_summary_stats()

Number of Users: 7568
Number of Utterances: 74467
Number of Conversations: 10744


## Bag-of-words prediction for utterances having positive scores

In [4]:
from convokit import BoWClassifier, BoWTransformer

In [5]:
bow_transformer = BoWTransformer(obj_type="utterance")

Initializing default unigram CountVectorizer...


In [6]:
print(next(corpus.iter_utterances()))

Utterance('id': 'nyx4d', 'root': nyx4d, 'reply-to': None, 'user': User('id': reddmau5, 'meta': {'num_posts': 3, 'num_comments': 5}), 'timestamp': 1325452698, 'text': "I was just reading about the Princeton Mic-Check and it's getting [national press](http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html).\n\nI want to get a sense of what people felt like around campus. Anything interesting happen? Anything interesting coming up?", 'meta': {'score': 2, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/nyx4d/so_i_was_away_this_past_semester_whats_going_on/', 'author_flair_text': ''})


In [8]:
bow_transformer.fit_transform(corpus)

In [9]:
print(next(corpus.iter_utterances()))

Utterance('id': 'nyx4d', 'root': nyx4d, 'reply-to': None, 'user': User('id': reddmau5, 'meta': {'num_posts': 3, 'num_comments': 5}), 'timestamp': 1325452698, 'text': "I was just reading about the Princeton Mic-Check and it's getting [national press](http://www.bloomberg.com/news/2011-12-29/princeton-brews-trouble-for-us-1-percenters-commentary-by-michael-lewis.html).\n\nI want to get a sense of what people felt like around campus. Anything interesting happen? Anything interesting coming up?", 'meta': {'score': 2, 'top_level_comment': None, 'retrieved_on': -1, 'gilded': -1, 'gildings': None, 'subreddit': 'Cornell', 'stickied': False, 'permalink': '/r/Cornell/comments/nyx4d/so_i_was_away_this_past_semester_whats_going_on/', 'author_flair_text': '', 'bow_vector': <1x9340 sparse matrix of type '<class 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>})


In [10]:
bow_classifier = BoWClassifier(obj_type="utterance", labeller=lambda utt: utt.meta['score'] > 0)

Initializing default classification model (standard scaled logistic regression)


In [11]:
bow_classifier.fit_transform(corpus)

In [12]:
next(corpus.iter_utterances()).meta

{'score': 2,
 'top_level_comment': None,
 'retrieved_on': -1,
 'gilded': -1,
 'gildings': None,
 'subreddit': 'Cornell',
 'stickied': False,
 'permalink': '/r/Cornell/comments/nyx4d/so_i_was_away_this_past_semester_whats_going_on/',
 'author_flair_text': '',
 'bow_vector': <1x9340 sparse matrix of type '<class 'numpy.int64'>'
 	with 42 stored elements in Compressed Sparse Row format>,
 'prediction': True,
 'pred_score': 0.9999998491105527}

In [13]:
next(corpus.iter_utterances()).meta['bow_vector'].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [14]:
bow_classifier.summarize(corpus).head()

,prediction,pred_score
id,,
dhhm9sa,True,1.0
dw553ml,True,1.0
dvzmhdx,True,1.0
dvzpp79,True,1.0
dw0imao,True,1.0


In [15]:
corpus.get_utterance('15enm8').text

'One, just to get this out of the way: I\'m only a sophomore in high school. In spite of this, my high school is one of the top public schools in New Jersey (and to put it bluntly it\'s a very affluent area... although I\'m not necessarily affluent like my classmates). The point of telling you guys that is kids start talking about all these amazing schools they want to go to in like eighth grade, so I know quite a bit about colleges. As stated in the title, I really want to go to Cornell, and I just was hoping that some of you guys and girls on here would be awesome enough to give out some SAT scores, ACT scores (if you took them), and extra curricular activities you guys got/did? My unweighted GPA is a 3.8 (weighted is a 4.2), and my first PSAT was an overall 1900, and from taking that I (not to sound cocky here) *know* that I\'m going to get that score up a *lot*. I\'m in all the highest level classes I can be in, and I\'m looking to take multiple AP courses next year (junior). Do yo

In [16]:
bow_classifier.get_coefs(feature_names=bow_transformer.get_vocabulary()).head()

,coef
feat_name,
hotels,1.270001
hbhs,1.115690
engine,1.109702
involves,1.081836
lincoln,1.071464


In [17]:
bow_classifier.get_coefs(feature_names=bow_transformer.get_vocabulary()).tail()

,coef
feat_name,
mahogany,-0.667785
ignoreme,-0.722992
hilton,-0.742234
binary,-0.764383
creation,-0.784593


In [18]:
y_true, y_pred = bow_classifier.get_y_true_pred(corpus)

In [19]:
bow_classifier.base_accuracy(corpus)

0.9279546644822538

In [20]:
bow_classifier.accuracy(corpus)

0.9491452589737737

In [21]:
print(bow_classifier.classification_report(corpus))

              precision    recall  f1-score   support

       False       0.88      0.34      0.49      5365
        True       0.95      1.00      0.97     69102

    accuracy                           0.95     74467
   macro avg       0.91      0.67      0.73     74467
weighted avg       0.95      0.95      0.94     74467



## Bag-of-words prediction for comment thread doubling in length versus staying the same length based on first 5 utterances

In [22]:
top_level_comment_ids = [utt.id for utt in corpus.iter_utterances() if utt.id == utt.meta['top_level_comment']]

In [23]:
corpus.print_summary_stats()

Number of Users: 7568
Number of Utterances: 74467
Number of Conversations: 10744


In [24]:
len(top_level_comment_ids)

32893

In [25]:
threads_corpus = corpus.reindex_conversations(new_convo_roots=top_level_comment_ids)


['c3p8bze', 'c3p1rn8', 'c3oyf4d', 'c3od15i', 'c3ocsyl']


In [26]:
threads_corpus.print_summary_stats()

Number of Users: 6160
Number of Utterances: 63697
Number of Conversations: 32888


In [27]:
for thread in threads_corpus.iter_conversations():
    thread_len = len(list(thread.iter_utterances()))
    if thread_len == 5:
        thread.meta['thread_doubles'] = False
    elif thread_len >= 10:
        thread.meta['thread_doubles'] = True
    else:
        thread.meta['thread_doubles'] = None

In [28]:
bow_transformer2 = BoWTransformer(obj_type="conversation", 
                    text_func=lambda convo: " ".join([utt.text for utt in convo.get_chronological_utterance_list()][:5]),
                   )

Initializing default unigram CountVectorizer...


In [29]:
bow_transformer2.fit_transform(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [30]:
bow_classifier2 = BoWClassifier(obj_type="conversation", 
                    labeller=lambda convo: convo.meta['thread_doubles'])

Initializing default classification model (standard scaled logistic regression)


In [31]:
bow_classifier2.fit_transform(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [32]:
summary = bow_classifier2.summarize(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [33]:
summary.head()

,prediction,pred_score
id,,
dt05qyf,True,1.0
dandio0,True,1.0
dwa6k96,True,1.0
dsldpxg,True,1.0
e70wjy3,True,1.0


In [34]:
summary.tail()

,prediction,pred_score
id,,
drduxx1,False,2.456000e-12
dl7q7n2,False,8.203326e-14
dxfib8r,False,2.715731e-15
dwqaa06,False,2.772276e-16
d8y9akn,False,1.673094e-16


In [35]:
bow_classifier2.base_accuracy(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

0.6761904761904762

In [36]:
bow_classifier2.accuracy(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

0.9992063492063492

In [37]:
print(bow_classifier2.classification_report(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00       852
        True       1.00      1.00      1.00       408

    accuracy                           1.00      1260
   macro avg       1.00      1.00      1.00      1260
weighted avg       1.00      1.00      1.00      1260



In [38]:
bow_classifier2.get_coefs(feature_names=bow_transformer2.get_vocabulary()).head(10)

,coef
feat_name,
nothing,0.611857
house,0.513383
common,0.508421
stop,0.481745
saw,0.463225
removed,0.456108
media,0.441341
profile,0.417163
500,0.414041


In [39]:
bow_classifier2.get_coefs(feature_names=bow_transformer2.get_vocabulary()).tail(10)

,coef
feat_name,
hill,-0.371359
added,-0.383444
tried,-0.383819
nice,-0.387260
70,-0.403524
god,-0.416965
joke,-0.419549
goes,-0.427090
enjoy,-0.491875


In [40]:
bow_classifier2.confusion_matrix(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

array([[852,   0],
       [  1, 407]])

## Paired bag-of-words prediction for comment thread doubling in length versus staying the same length based on first 5 utterances

In [41]:
from convokit import Pairer, PairedBoW

In [42]:
corpus = Corpus(filename=download('subreddit-Cornell'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/subreddit-Cornell


In [43]:
threads_corpus = corpus.reindex_conversations(new_convo_roots=top_level_comment_ids, preserve_convo_meta=True)


['c3p8bze', 'c3p1rn8', 'c3oyf4d', 'c3od15i', 'c3ocsyl']


In [46]:
next(threads_corpus.iter_conversations())

Conversation({'obj_type': 'conversation', '_owner': <convokit.model.corpus.Corpus object at 0x12e9aba10>, 'meta': {'original_convo_meta': {'title': 'Any upcoming transfer students out there on /r/cornell?', 'num_comments': 11, 'domain': 'self.Cornell', 'timestamp': 1366243785, 'subreddit': 'Cornell', 'gilded': 0, 'gildings': None, 'stickied': False, 'author_flair_text': ''}, 'original_convo_id': '1ckkpf', 'thread_doubles': None}, '_id': 'c9heblf', '_utterance_ids': ['c9heblf'], '_user_ids': None, 'tree': None})

In [45]:
for thread in threads_corpus.iter_conversations():
    thread_len = len(list(thread.iter_utterances()))
    if thread_len == 5:
        thread.meta['thread_doubles'] = False
    elif thread_len >= 10:
        thread.meta['thread_doubles'] = True
    else:
        thread.meta['thread_doubles'] = None

In [47]:
next(threads_corpus.iter_conversations()).meta

{'original_convo_meta': {'title': 'Any upcoming transfer students out there on /r/cornell?',
  'num_comments': 11,
  'domain': 'self.Cornell',
  'timestamp': 1366243785,
  'subreddit': 'Cornell',
  'gilded': 0,
  'gildings': None,
  'stickied': False,
  'author_flair_text': ''},
 'original_convo_id': '1ckkpf',
 'thread_doubles': None}

In [48]:
pairer = Pairer(obj_type="conversation", 
                pos_label_func=lambda convo: convo.meta['thread_doubles'], 
                neg_label_func=lambda convo: convo.meta['thread_doubles'] == False,
                pairing_func=lambda convo: convo.meta['original_convo_id']
               )

In [49]:
pairer.transform(threads_corpus, selector=lambda convo: convo.meta['thread_doubles'] is not None)

In [50]:
print(next(threads_corpus.iter_conversations()))

Conversation('id': 'c9heblf', 'utterances': ['c9heblf'], 'meta': {'original_convo_meta': {'title': 'Any upcoming transfer students out there on /r/cornell?', 'num_comments': 11, 'domain': 'self.Cornell', 'timestamp': 1366243785, 'subreddit': 'Cornell', 'gilded': 0, 'gildings': None, 'stickied': False, 'author_flair_text': ''}, 'original_convo_id': '1ckkpf', 'thread_doubles': None, 'pair_obj_label': None, 'pair_id': None, 'pair_orientation': None})


In [51]:
for convo in threads_corpus.iter_conversations():
    if convo.meta['pair_id'] is not None:
        print(convo)
        break

Conversation('id': 'dos8twc', 'utterances': ['dos8twc', 'doscuvh', 'dosgrxp', 'doxr7u3', 'doxra3t'], 'meta': {'original_convo_meta': {'title': 'Anti-Semitic Posters Appear on Cornell’s Campus', 'num_comments': 22, 'domain': 'cornellsun.com', 'timestamp': 1508771001, 'subreddit': 'Cornell', 'gilded': 0, 'gildings': None, 'stickied': False, 'author_flair_text': ''}, 'original_convo_id': '788atj', 'thread_doubles': False, 'pair_obj_label': 'neg', 'pair_id': '788atj', 'pair_orientation': 'pos'})


In [52]:
from convokit import BoWTransformer

In [53]:
bow_transformer = BoWTransformer(obj_type="conversation", 
                                 text_func=lambda convo: " ".join([utt.text for utt in convo.get_chronological_utterance_list()][:5]),
                    )

Initializing default unigram CountVectorizer...


In [54]:
bow_transformer.fit_transform(threads_corpus, selector=lambda convo: convo.meta['pair_id'] is not None)

In [55]:
for convo in threads_corpus.iter_conversations():
    if convo.meta['pair_id'] is not None:
        print(convo)
        break

Conversation('id': 'dos8twc', 'utterances': ['dos8twc', 'doscuvh', 'dosgrxp', 'doxr7u3', 'doxra3t'], 'meta': {'original_convo_meta': {'title': 'Anti-Semitic Posters Appear on Cornell’s Campus', 'num_comments': 22, 'domain': 'cornellsun.com', 'timestamp': 1508771001, 'subreddit': 'Cornell', 'gilded': 0, 'gildings': None, 'stickied': False, 'author_flair_text': ''}, 'original_convo_id': '788atj', 'thread_doubles': False, 'pair_obj_label': 'neg', 'pair_id': '788atj', 'pair_orientation': 'pos', 'bow_vector': <1x353 sparse matrix of type '<class 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>})


In [56]:
paired_bow = PairedBoW(obj_type="conversation")

In [57]:
paired_bow.fit(threads_corpus)

Found 65 valid pairs.


In [58]:
paired_bow.summarize(threads_corpus)

Found 65 valid pairs.


0.47692307692307695

In [59]:
paired_bow.get_coefs(feature_names=bow_transformer.get_vocabulary())

,coef
feat_name,
shit,0.391323
school,0.338156
similar,0.307523
accepted,0.306597
computer,0.257602
...,...
sounds,-0.268842
each,-0.276740
did,-0.382643
